In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Transformation Interface
To continue, use the corpus that is saved in session Corpora and Vector Spaces

In [2]:
from gensim import corpora, models, similarities
import os
os.chdir("tmp")
dictionary = corpora.Dictionary.load('deerwester.dict')
corpus = corpora.MmCorpus('deerwester.mm')
print(corpus)

MmCorpus(9 documents, 12 features, 28 non-zero entries)


Two goals to transform documents from one vector representation into another:
- 1. To bring out hidden structure in the corpus, discover relationships between words and use them to describe the documents in a new and more semantic way.
- 2. To make the document representation more compact. This both improves efficiency (new representation consumes less resources) and efficacy (marginal data trends are ignored, noise-reduction)

### Creating a Transformation
The transformations are standard Python objects, typically initialized by means of a training corpus:

In [3]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize(train) a model

### Transforming Vectors
From now on, tfidf is treated as a read-only object that can be used to convert any vector from the old representation (bag-of-words integer counts) to the new representation (Tf-Idf real-valued weights):

In [4]:
doc_bow = [(0,1), (1,1)]
print(tfidf[doc_bow]) # step 2 -- use the model to transform vectors

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


Or to apply a transformation to a whole corpus:

In [5]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(1, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.44424552527467476), (6, 0.3244870206138555), (7, 0.3244870206138555)]
[(0, 0.5710059809418182), (6, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(2, 0.49182558987264147), (6, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (4, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(5, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


Transformations can also be serialized, one on top of another, in a sort of chain. 
Now transform Tf-Idf corpus via Latent Semantic Indexing into a latent 2-D space.

In [6]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformation are executed on the fly
    print(doc)

[(0, 0.066007833960900486), (1, -0.52007033063618602)]
[(0, 0.19667592859142116), (1, -0.76095631677000597)]
[(0, 0.089926399724460301), (1, -0.72418606267525176)]
[(0, 0.075858476521778517), (1, -0.63205515860034378)]
[(0, 0.10150299184979832), (1, -0.57373084830029608)]
[(0, 0.70321089393783154), (1, 0.16115180214025407)]
[(0, 0.87747876731198371), (1, 0.16758906864658951)]
[(0, 0.9098624686818586), (1, 0.14086553628718559)]
[(0, 0.61658253505692828), (1, -0.053929075663896556)]


Inspect the two latent dimensions:

In [7]:
lsi.print_topics(2)

[u'0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"',
 u'-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"']

Model persistency is achieved with the save() and load() functions:

In [8]:
lsi.save('model.lsi') # same for tfidf, lda, ...
lsi = models.LsiModel.load('model.lsi')

### Available Transformations
Gensim implements several popular VSM algorithms

#### Term Frequency - Inverse Document Frequency (Tf-Idf):
It converts integer-valued vectors into real-valued ones, while leaving the number of dimensions intact. It can also optionally normalize the resulting vectors to unit length.

Usage:

    model = tfidfmodel.TfidfModel(bow_corpus, normalize=True)

#### Latent Semantic Index (LSI or LSA):
It transforms documents from either bag-of-words or (preferrably) Tf-Idf weighted space into a latent space of a lower dimensionality. On real corpora, target dimensionality of 200-500 is recommended as a "golden standard".

Usage: 

    model = lsimodel.LsiModel(tfidf_corpus, id2word=dictionary, num_topics=300)

LSI training is unique in that we can continue training at any point, simply by providing more training documents. This is done by incremental updates to the underlying model, called online training. Because of this feature, the input document stream may even be infinite - just keep feeding LSI new documents as they arrive, while using the computed transformation model as read-only in the meanwhile!

Usage:

    model.add_documents(another_tfidf_corpus) # now LSI has been trained on tfidf_corpus + another_tfidf_corpus
    lsi_vec = model[tfidf_vec] # convert some new document into the LSI space, without affecting the model
    model.add_document(more_documents) 
    lsi_vec = model[tfidf_vec]

#### Random Projections (RP):
It aims to reduce vector space dimensionality, by throwing in a little randomness. Recommended target dimensionality is in hundreds/thousands.

Usage:
    
    model = rpmodel.RpModel(tfidf_corpus, num_topics=500)

#### Latent Dirichlet Allocation (LDA):
It is a probabilistic extension of LSA.

Usage:
    
    model = ldamodel.LdaModel(bow_corpus, id2word=dictionary, num_topics=100)

In [9]:
os.chdir("../")
print("Done!")

Done!
